In [89]:
import glob
import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.nn.functional as F
import tqdm

import time
import os

In [90]:
#load dataset from npy file
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt




In [91]:
args={}
kwargs={}
args['batch_size']=64
args['test_batch_size']=64
args['epochs']=40  #The number of Epochs is the number of times you go through the full dataset. 
args['lr']=0.01 #Learning rate is how fast it will decend. 
args['momentum']=0.5 #SGD momentum (default: 0.5) Momentum is a moving average of our gradients (helps to keep direction).

args['seed']=1 #random seed
args['log_interval']=100 // args['batch_size']
args['cuda']=False


In [92]:
data = np.load('trainset.npy')
# data = torch.from_numpy(data)
#data = data.permute(0,3,1,2)
label = np.load('trainlabel.npy')
# label = torch.from_numpy(label)
test = np.load('testset.npy')
# test = torch.from_numpy(test)
#test = test.permute(0,3,1,2)




In [93]:
#make dataset with data and label
class MyDataset(Dataset):

    def __init__(self, data, label,train = True,transform= None):
        
        if train:
            self.data = data
            self.label = label
        else:
            self.data = test
            self.label = None

        self.transform = transform

         
        
    def __getitem__(self, index):
        data = self.data[index]    
        if self.transform:
            self.data = self.transform(self.data)

        if self.label is not None:
            label = self.label[index]
            return data, label
        else:
            return data
    def __len__(self):
        return len(self.data)

    

In [94]:
class testDataset(Dataset):

    def __init__(self, data, transform= None):
        
        self.data = test
        self.label = None
        self.transform = transform

    def __getitem__(self, index):
        data = self.data[index]    
        return data
    def __len__(self):
        return len(self.data)

In [95]:
transform_train = transforms.Compose([
    transforms.ToTensor(),
    # transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])
transform_test = transforms.Compose([
    # transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

trainset = MyDataset(data, label,train = True,transform= transform_train)
trainset , valset = torch.utils.data.random_split(trainset, [40000, 10000])
testset = testDataset(data,transform= transform_test)
trainloader = DataLoader(trainset, batch_size= args['batch_size'], shuffle= True)
valloader = DataLoader(valset, batch_size= args['batch_size'], shuffle= True)
testloader = DataLoader(testset, batch_size= args['batch_size'], shuffle= False)


In [96]:
# #check the dataset
# print(len(trainset))
# print(len(valset))
# print(len(testset))
# print(len(trainloader))
# print(len(valloader))
# print(len(testloader))
# print()


# data, label = next(iter(trainloader))
# data, label
# print()

# #check the trainset images with label
# for i in range(5):
#     data,label = trainset[i]
#     print(data.shape)
#     print(label)
#     plt.imshow(torch.permute(data, (1, 2, 0)), cmap='gray')
#     plt.show()





40000
10000
10000
625
157
157



ValueError: pic should be 2/3 dimensional. Got 4 dimensions.

    x = x.permute(0,3,1,2)####

In [ ]:
#32*32*3 convolutional neural network (two convolutional layers)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 100)

    def forward(self, x):
        print(x.shape)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x




In [ ]:
for batch, (X,y)in enumerate(trainloader):
    print(batch)
    print(X.shape)
    print(y.shape)
    break

0
torch.Size([64, 32, 32, 3])
torch.Size([64])


In [ ]:

def train_loop(dataloader, model, loss_fn, optimizer):
    model.train()
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X.float())
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 128 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X.float())
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return correct


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

model = Net().to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)

Using cpu device


In [ ]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
pytorch_total_params

63092

In [ ]:

base_correct = 0
for t in range(args['epochs']):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(trainloader, model, loss_fn, optimizer)
    correct = test_loop(valloader, model, loss_fn)
    if correct > base_correct:
        print('save model')
        torch.save(model.state_dict(), 'model3.pt')
        base_correct = correct
print("Done!")

Epoch 1
-------------------------------


RuntimeError: Given groups=1, weight of size [6, 3, 5, 5], expected input[64, 32, 32, 3] to have 3 channels, but got 32 channels instead

In [ ]:
import csv

In [ ]:
def test(dataloader, model):
    idx = 0
    result = []
    model.eval()
    with torch.no_grad():
        for X in dataloader:
            X = X.to(device)
            outputs = model(X.float())
            _, predicted = torch.max(outputs.data, 1)

            try:
                for i in range(batch):
                    result.append([idx,predicted[i].item()])
                    idx+=1
            except:
                pass
    return result

model.load_state_dict(torch.load('model3.pt'))
result = test(testloader,model)
fields = ['id_idx','label']
with open('result.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(fields)
    writer.writerows(result)

RuntimeError: Input type (unsigned char) and bias type (float) should be the same